In [ ]:
get_my_data('2019-01-01')

1


In [ ]:
import ray
ray.init()
# ray.shutdown()

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)

import pandas_bokeh
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

import polygon_ds as pds
import polygon_df as pdf
import polygon_s3 as ps3
import ray_symbol_details as rsd
import market_preprocess as mp
import market_cov as mcov
import market_cluster as mc
import utils_pickle as up
# import ipdb
# ipdb.set_trace(context=10)

In [ ]:
start_date = '2018-10-24'
end_date = '2020-11-24'
symbol = 'market'
tick_type='daily'

In [ ]:
r = mp.prepare_data(start_date, end_date, beta_symbol='VTI')

In [ ]:
X = r['g_zs_log_returns_resid'].transpose()
gt_industry = r['sym_meta'].industry.values
gt_sector = r['sym_meta'].sector.values

In [ ]:
# r['g_zs_log_returns_resid'].iloc[:, 1500:]
dist_mat, cor_mat = mcov.cov_denoise_detone_dist(r['g_zs_log_returns'], detone=True)

In [ ]:
X = dist_mat
kclust = mc.get_cluster_evaluation(mc.get_kmean_clusters, 200, X, gt_industry)
hclust = mc.get_cluster_evaluation(mc.get_hira_clusters, 200, X, gt_industry)

In [ ]:
mc.evaluate_clustering(kclust['cluster_labels'], hclust['cluster_labels'])

In [ ]:
kmeadist_mat                             hclust
-------------------------          ----------------------------
X g zs log rets                     X g zs log rets: 435
X resids: 45                        X resids: 464
X resids dncor dist: 34
X resids corr: 33
X resids dncor detone: 37           X resids dncor detone: 386

In [ ]:
corex300 = up.pickle_load('data/corex300.pkl')

# X = df.reset_index(drop=True)
# corex, full_df = mc.corex_fit(X, n_hidden=200)

# up.pickle_dump((corex, full_df), file_name='corex200.pkl')

# gsutil copy /Users/bobcolner/QuantClarity/pandas-polygon/corex200.pkl gs://emerald-skill-datascience/

In [ ]:
import h2o
from h2o.estimators import H2OGeneralizedLowRankEstimator
h2o.init()

# Import the USArrests dataset into H2O:
arrestsH2O = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/pca_test/USArrests.csv")

# Split the dataset into a train and valid set:
train, valid = arrestsH2O.split_frame(ratios=[.8], seed=1234)

# Build and train the model:
glrm_model = H2OGeneralizedLowRankEstimator(
    k=4,
    loss="Huber",
    loss_by_col={'cat_var': 'Categorical'},
    regularization_x="OneSparse", # None (default), Quadratic, L2, L1, NonNegative, OneSparse, UnitOneSparse, Simplex.
    regularization_y="OneSparse",
    gamma_x=0.5,
    gamma_y=0.5,
    max_iterations=1000,
    recover_svd=True,
    init="PlusPlus",
    transform="standardize",
   )
glrm_model.train(training_frame=train)